# REST APIs

Today's lesson is all about REST APIs - if these letters mean nothing to you yet, strap in.  

In day to day development we often come across problems that someone else already solved. This
deep in the course you all are aware that reinventing the wheel is never a great idea.  

While many solutions can be accessed by importing libraries, others are provided via Application
Programming Interfaces, or APIs. Often, APIs are used to gain access to data of all sorts. This
means **C**reating, **R**eading, **U**pdating and **D**eleting data. We refer to these four atomic operations
as CRUD operations.

A special type of API is the REST API, or Representational State Transfer API. These well defined
APIs are a staple in software development. You will for example find that all popular cloud providers
offer a REST API to give programmatic access to resources.

The great benefit of these APIs is the simplicity of them. While other data sources might require
you to install countless additional packages, SDKs and so on, a REST API is just a web service
running in the cloud or in an on-premises data center. We access these APIs with HTTP requests
and the data exchange is done via JSON (Javascript Object Notation).

Consuming an API endpoint can be done with even the simplest of tools. All you need is
a way of invoking an URL, optionally with some data. With Python, this ability
is introduced by means of the `requests` module.

So, before continuing to run the examples, be sure to install this library!  
```bash
pip install requests
```

## Accessing an API

Before we delve into Python, we need to examine APIs a bit more. Specifically the URL format
used by all APIs, as well as the different ways of passing parameters to these APIs.  

Across all providers, APIs usually consist of multiple endpoints. A Weather API might for example
contain seperate API endpoints to retrieve either the forecast, or the current weather, or
specific weather information like aeronautical or agricultural weather.

Generally speaking, each endpoint is used to manipulate one type of objects. We can usually
infer the object type roughly by just looking at the URL.

```output
/posts      100 posts
/comments   500 comments
/albums     100 albums
/photos     5000 photos
/todos      200 todos
/users      10 users
```  

In the previous example, the endpoint names already make it clear what we can expect to get back from each endpoint.
The users endpoint for example will let you access up to 10 user objects, however they may
look like.

Since we access REST APIs through HTTP, we use different HTTP verbs in order to interact with the
endpoint. The same endpoint will behave differently depending on the verb used. Reading data for example
is obviously different to updating data.

For our next operations we will use a fake API you can test against, hosted at <https://jsonplaceholder.typicode.com>.
This API allows us to test all operations by simulating the result. We will start with the R in CRUD, Read, before
we start to manipulate the data returned by the API.

### Read (GET)

The HTTP verb GET is used to do read operations against an API. In our next sample, we try the unbounded
read first. The API endpoint, as its documentation states, returns 10 user objects. While we have no clue
as to how they are stored we do know the data type we get: A dictionary.

The data used within the API likely exists as some type of object, but everytime we hit the API the
objects are serialized to a JSON string. They are quite literally deconstructed, sent over the wire
and reconstructed or deserialized on the other end.

Not all data types are easy to serialize, but this should not be of concern when you are consuming
the API. When developing your own API though this is something to watch out for.

In [ ]:
import requests
response = requests.get('https://jsonplaceholder.typicode.com/users')

# The response contains a response code of 200, indicating a success
# This means we can decode the JSON reply of the API back to a dictionary
if response.status_code == requests.codes.ok:
    for r in response.json():
        print('Found user ID ' + str(r['id']) + ' belonging to ' + r['name'])
        
        # As with any dictionary, you can freely access nested keys and values
        print('This user lives in ' + r['address']['city'])

With every request, we get a list of objects back. Filtering them after having essentially
downloaded everything has a very bad performance. As a general rule, try to filter as
early as possible. In this case, the API can filter for us before returning the data
to our program.

In [ ]:
# The ID carries a special meaning, we can use it directly as part of the URL
response = requests.get('https://jsonplaceholder.typicode.com/users/1')
deserializedResponse = response.json()
print('Retrieved single user ' + deserializedResponse['name'])

In [ ]:
# We can do a similar operation using a Query string
# Notice that the result of a Query is a list, even if it only contains one object
response = requests.get('https://jsonplaceholder.typicode.com/users?id=1')
deserializedResponse = response.json()
print('Retrieved single user ' + deserializedResponse[0]['name'])

In [ ]:
# The query string can be very flexible if implemented in the API
response = requests.get('https://jsonplaceholder.typicode.com/users?name=Leanne%20Graham')
deserializedResponse = response.json()
print('Retrieved single user ' + deserializedResponse[0]['name'])

### Create (PUT)

The first method in our CRUD methods is Create which is implemented using the HTTP verb PUT.
The data for the object that is created by the PUT verb is passed either using a Query string
or via the request body. APIs often allow both, but refer to the documentation of the API first.  

To continue with our example, we want to create a new user, with some additional properties set  
like the address of the account. To this end, we create a new dictionary that contains the
necessary data.


In [ ]:
user = {
  "name": "Bobby Tables",
  "username": "robert_tables",
  "company": {
      "name" : "A Company"
  }
}
print(user)
response = requests.post('https://jsonplaceholder.typicode.com/users', data = user)
print(response.json()['name'])

### Update (POST)
### Delete (DELETE)